In [ ]:
import os
from src.rds_vdb.document import Document
from src.rds_vdb.embedder.hf_embedder import HuggingFaceEmbedder
from src.rds_vdb.backends.rds_oracle_backend import RDSOracleVectorBackend
import traceback


print("--- INICIANDO TESTE COMPLETO ---")


DB_ENDPOINT = "" 
DB_PASSWORD = ""             

MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'
DIMENSION = 384

db_config = {
  "user": "admin",
  "password": DB_PASSWORD,
  "host": DB_ENDPOINT,
  "port": 1521,
  "service_name": "ORCL",
  "table": "",
  "dim": DIMENSION,
  "ensure_schema": True
}

db = None 

try:
   
    print("\n[PASSO 2] Carregando modelo de embedding...")
    embedder = HuggingFaceEmbedder(model_name=MODEL_NAME)
    print(f"-> Modelo '{MODEL_NAME}' carregado (Dimensão: {embedder.dim}).")

    print("\n[PASSO 2] Conectando ao Oracle RDS...")
    db = RDSOracleVectorBackend(db_config)
    if db.is_open():
        print("-> Conexão com o Oracle RDS estabelecida com sucesso!")
    else:
        raise Exception("Falha ao estabelecer conexão com o Oracle RDS.")

    textos_para_inserir = [
        "A Torre Eiffel está localizada em Paris, na França.",
        "O sol é a estrela no centro do nosso sistema solar.",
        "A fotossíntese é o processo usado pelas plantas para converter luz em energia.",
        "O Brasil é o maior país da América do Sul e sua capital é Brasília."
    ]
    
    print(f"\n[PASSO 3] Gerando embeddings para {len(textos_para_inserir)} documentos...")
    embeddings = embedder.embed(textos_para_inserir)
    print("-> Embeddings gerados.")

    docs = [
        Document(page_content=text, embedding=emb, metadata={"source": f"doc_{i}"})
        for i, (text, emb) in enumerate(zip(textos_para_inserir, embeddings))
    ]

    print(f"[PASSO 3] Inserindo {len(docs)} documentos no banco de dados...")
    db.insert(docs)
    print(f"-> Documentos inseridos com sucesso na tabela '{db_config['table']}'!")

 
    query_text = "Qual a capital do país sul-americano que fala português?"
    
    print(f"\n[PASSO 4] Consultando por: '{query_text}'")
    query_embedding = embedder.embed([query_text])[0]
    results = db.query(embedding=query_embedding, k=2)

    print("\n--- RESULTADOS DA BUSCA ---")
    if not results:
        print("Nenhum resultado encontrado.")
    for i, result in enumerate(results, 1):
        print(f"#{i} | Score: {result.score:.4f} (menor é melhor)")
        print(f"  Conteúdo: {result.doc.page_content}")
        print(f"  Metadados: {result.doc.metadata}\n")

except Exception as e:
    print("\n--- !!! OCORREU UM ERRO !!! ---")
    print(f"Erro: {e}")
    traceback.print_exc()

finally:
   
    if db and db.is_open():
        print("[PASSO 5] Fechando a conexão com o banco de dados.")
        db.close()
        print("-> Conexão fechada.")
    
    print("\n--- TESTE FINALIZADO ---")